In [3]:
##### TODO : mettre dans installation
!pip install fuzzywuzzy
!pip install python-Levenshtein
#erreur sans ce package /opt/conda/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
!pip install langdetect


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 41.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=be63f255ee53ed0ef2f5ef7a9e9479c4fcfc4c762560979f1f78c33e5fb0d0e3
  Stored in directory: /home/onyxia/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [4]:
##### TODO : mettre dans les imports
import pandas as pd
import requests
import time
from urllib.parse import quote
from langdetect import detect
from fuzzywuzzy import fuzz 


La deuxième étape de l'extraction des données consiste à trouver les thèmes des livres dont on sait qu'on dispose des textes dans Gutenberg, comme Gutenberg ne met pas à disposition cette information. On utilise donc l'API OpenLibrary, qui est l'une des rares API regroupant les thèmes des livres.
Pour se faire, on définit préalablement une liste d'auteurs à partir des [pages _wikipedia_ recensant les romanciers français](https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:Romancier_fran%C3%A7ais_par_si%C3%A8cle) que l'on croise avec nos connaissances de prépa B/L pour garder uniquement les auteurs dits "classiques". Cette démarche à trois avantages. Le premier est de maximiser les chances de trouver les thèmes des livres, puisque ces auteurs ont écrit des livres qui se sont inscrits dans la postérité. En d'autres termes, on évite de faire une trop grosse requête par souci environmental et par souci de temps nécessaire pour requêter l'API. Le deuxième est de ne pas disposer d'une base de données trop grande puisque les données deviendraient rapidement trop lourdes à traiter .Le troisième est de disposer d'une liste de livres facilement accessible dans la vie réelle selon





....... PORTEE DU RPROJET EN MODE RECOMMANDATION OU AUTRE CHOSE .............................

In [5]:
# List of French Writers abritrarily defined and chosen in the 17th, 18th and 19th century
authors = [
    # 17th century
    "Honoré d'Urfé", "Madeleine de Scudéry", "Paul Scarron", "Jean de La Fontaine", "Madame de Lafayette", "Charles Sorel", "Tristan L'Hermite", "François de Salignac de La Mothe-Fénelon",
    "Savinien de Cyrano de Bergerac",
    
    # 18th century
    "Montesquieu", "Voltaire", "Jean-Jacques Rousseau", "Denis Diderot", "Marivaux", "Abbé Prévost", "Pierre Choderlos de Laclos", "Beaumarchais", 
    
    # 19th century
    "Honoré de Balzac", "Victor Hugo", "Alexandre Dumas", "Gustave Flaubert", "Émile Zola", "Stendhal", "Alfred de Musset", "George Sand", "Jules Verne", "Alphonse Daudet", 
    "Théophile Gautier", "Edmond de Goncourt", "Joris-Karl Huysmans", "Octave Mirbeau", "Prosper Mérimée", "Eugène Sue", "Charles Nodier", "Gaston Leroux", 
    "François-René de Chateaubriand", "Anatole France", "Gustave Flaubert", "Alfred Jarry", "Guy de Maupassant", "Romain Rolland", "Alfred Séguin", "Alfred de Vigny", "Paul de Kock"
]

# Create a DataFrame
df = pd.DataFrame(authors, columns=["Authors"])

df


,Authors
0,Honoré d'Urfé
1,Madeleine de Scudéry
2,Paul Scarron
3,Jean de La Fontaine
4,Madame de Lafayette
5,Charles Sorel
6,Tristan L'Hermite
7,François de Salignac de La Mothe-Fénelon
8,Savinien de Cyrano de Bergerac
9,Montesquieu


In [6]:
#TODO : définir comment stocker ca
data = pd.read_csv('livres_fr_triés.csv')
data

,Description,Index
0,"La Comédie humaine, Vol. 17, Études de mœurs, ...",74126
1,Œuvres complètes de Guy de Maupassant - volume...,73633
2,"La Comédie humaine, Volume 16, Études philosop...",73552
3,"Œuvres complètes de Victor Hugo, by Victor Hug...",72885
4,"La Comédie humaine, Volume XV, Études philosop...",72034
...,...,...
471,"Le Tour du Monde en 80 Jours, by Jules Verne ...",800
472,"De La Terre a La Lune, by Jules Verne ...",799
473,"Le Rouge et le Noir, by Stendhal ...",798
474,"L'Abbesse de Castro, by Stendhal ...",797


On commence par nettoyer ce _data frame_.

In [8]:
# Séparation de la colonne en deux
data[['Title', 'Author']] = data['Description'].str.split(' by ', expand=True)

# Suppression de l'ancienne colonne si nécessaire
df_books = data.drop(columns=['Description'])
df_books

,Index,Title,Author
0,74126,"La Comédie humaine, Vol. 17, Études de mœurs,",Honoré de Balzac \n [Subtitle: La cousi...
1,73633,Œuvres complètes de Guy de Maupassant - volume...,Guy de Maupassant \n [Language: French]
2,73552,"La Comédie humaine, Volume 16, Études philosop...",Honoré de Balzac\n [Language: French]
3,72885,"Œuvres complètes de Victor Hugo,",Victor Hugo \n [Subti...
4,72034,"La Comédie humaine, Volume XV, Études philosop...",Honoré de Balzac\n [Language: French]
...,...,...,...
471,800,"Le Tour du Monde en 80 Jours,",Jules Verne \n [...
472,799,"De La Terre a La Lune,",Jules Verne ...
473,798,"Le Rouge et le Noir,",Stendhal ...
474,797,"L'Abbesse de Castro,",Stendhal ...


In [ ]:
url_api = "https://openlibrary.org/search.json?"

# Function to get the title of the books and the themes
def get_random_book(author):
    url  = url_api + f'q=author:{author}' + '&fields=title,first_publish_year,subject'
    req = requests.get(url)
    time.sleep(2)
    # Check if the request worked
    if req.status_code == 200:
        books = req.json().get("docs", [])
        index = df[df["Authors"] == author].index[0] 
        
        books_with_themes = [
            book for book in books
            if book.get("subject")  # Check if subjects are defined
            and len(book["subject"]) >= 5  
            and book.get("first_publish_year")  # Check if the release date is available 
            and isinstance(book["first_publish_year"], int)  # Check if the release date is an int
            and df["Birth"][index] <= book["first_publish_year"]  # Check if the book was published after the birth of the author to avoid problems of namesake
            and book["first_publish_year"] <= df["Death"][index]  # Check if the book was published before death
        ] 
        num_books = len(books_with_themes)
        if num_books > 0:
            # We select the maximum number of books after filtering
            selected_books = books_with_themes
            book_info = []
            for info in selected_books:
                title = info.get("title", "N/A")
                themes = info.get("subject", "N/A")
                year = info.get("first_publish_year", "N/A")
                book_info.append((author, title, year, themes))
            return book_info
        else:
            return [(author, '', '','')]
    else:
        return [(author, '', '','')]
    


In [ ]:
list_books = []
for author in df["Authors"]:
    list_books.append(get_random_book(author))


[]
0

[{'first_publish_year': 1691, 'title': 'Artamène', 'subject': ['Court and courtiers', 'Fiction', 'Social conditions', 'Social life and customs', 'Women', 'Fiction, general', 'France, fiction']}]
1

[{'first_publish_year': 1649, 'title': 'Svr la conference de Rvel en mars', 'subject': ['Fronde', 'Poetry', 'History', 'France. 1649 March 11', 'France']}]
1

[{'first_publish_year': 1678, 'title': 'Fables', 'subject': ['French Fables', 'Translations into English', 'Fables', 'Oversize books', 'Illustrations', 'Adaptations', 'Translations into Malagasy', 'Specimens', "Aesop's fables", 'English Fables', 'French language', 'Translations into French Creole', 'Readers', 'Poetry', 'French language materials', 'Translations into Yiddish', 'Juvenile literature', 'Translations from French', 'Translations into Arabic', 'Translations into Esperanto', 'Translations into Occitan', 'Toy and movable books', 'Dialects', 'Latin language', 'Yiddish Fables', 'Translations into Italian', 'Translations', '

In [47]:
# As we have a : list[list[tuples]], we convert it to list[tuples] to transform it to a df afterward
flattened_books = []
for author_data in list_books:
    for book in author_data:
        flattened_books.append(book)

# Convertir en DataFrame
df_books = pd.DataFrame(flattened_books, columns=['Author', 'Title', 'Year', 'Themes'])


df_books


,Author,Title,Year,Themes
0,Honoré d'Urfé,,,
1,Madeleine de Scudéry,Artamène,1691,"[Court and courtiers, Fiction, Social conditio..."
2,Paul Scarron,Svr la conference de Rvel en mars,1649,"[Fronde, Poetry, History, France. 1649 March 1..."
3,Jean de La Fontaine,Fables,1678,"[French Fables, Translations into English, Fab..."
4,Jean de La Fontaine,Contes et nouvelles en vers,1685,"[Translations into English, Translations into ..."
...,...,...,...,...
188,Guy de Maupassant,,,
189,Romain Rolland,Musiciens d'aujourd'hui,1800,"[Music, History and criticism, Musicians, Biog..."
190,Alfred Séguin,,,
191,Alfred de Vigny,,,


In [48]:
# delete lines of the DF for which no book was found
df_books = df_books[df_books['Title'] != '']
df_books.reset_index(drop=True, inplace=True)


In [49]:
# Transform the list of themes in a string
df_books['Themes'] = df_books['Themes'].apply(lambda x: ', '.join(x))
df_books

/tmp/ipykernel_12013/1577876212.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_books['Themes'] = df_books['Themes'].apply(lambda x: ', '.join(x))


,Author,Title,Year,Themes
0,Madeleine de Scudéry,Artamène,1691,"Court and courtiers, Fiction, Social condition..."
1,Paul Scarron,Svr la conference de Rvel en mars,1649,"Fronde, Poetry, History, France. 1649 March 11..."
2,Jean de La Fontaine,Fables,1678,"French Fables, Translations into English, Fabl..."
3,Jean de La Fontaine,Contes et nouvelles en vers,1685,"Translations into English, Translations into O..."
4,François de Salignac de La Mothe-Fénelon,Aventures de Télémaque,1699,"Fiction, Telemachus (Greek mythology), Princes..."
...,...,...,...,...
175,Anatole France,The crime of Sylvestre Bonnard (member of the ...,1890,"Philologists, Men, Book collectors, Children a..."
176,Gustave Flaubert,Madame Bovary,1857,"literary fiction, French literature, Fiction, ..."
177,Gustave Flaubert,Salammbô,1863,"Fiction, History, Continental european fiction..."
178,Gustave Flaubert,Trois contes,1877,"Classic Literature, Fiction, French language, ..."


In [ ]:
url = "https://lingva.ml/api/v1/"
source = 'auto/' # auto as some themes may be written in another language than English
dest = 'fr/'
for i in range(len(df_books['Title'])):
    if detect(df_books['Title'][i]) != 'fr':
        url_api_transl = url + source + dest + quote(df_books['Title'][i])
        response = requests.get(url_api_transl)
        translated_themes = response.json().get("translation", "")
        df_books.loc[i,'Title'] = translated_themes
        time.sleep(2) # The API limit the flow of request


In [33]:
def remove_similar_books_from_df(df, threshold=80):
    to_remove = []  # Liste pour enregistrer les indices des livres à supprimer

    # Grouper les livres par auteur
    list_books_per_author = df.groupby('Author')['Title'].apply(list)

    # Parcourir chaque groupe d'auteur
    for author, books in list_books_per_author.items():
        # Comparer tous les titres dans chaque groupe
        for i in range(len(books)):
            for j in range(i + 1, len(books)):  # Comparer un titre avec tous les autres après lui
                similarity = fuzz.ratio(books[i], books[j])
                if similarity >= threshold:
                    # Si les titres sont similaires, ajouter l'indice à supprimer
                    index_to_remove = df[(df['Author'] == author) & (df['Title'] == books[j])].index
                    to_remove.extend(index_to_remove)  # Ajouter l'indice à la liste

    # Supprimer les lignes contenant les indices marqués
    df = df.drop(to_remove).reset_index(drop=True)
    
    return df

In [56]:
df_books = remove_similar_books_from_df(df_books)

In [59]:
# The goal is now to clean the themes to only keep relevant themes, and to translate them into French
# To translate themes, we use the API Lingva

url = "https://lingva.ml/api/v1/"
source = 'auto/' # auto as some themes may be written in another language than English
dest = 'fr/'

for i in range(len(df_books['Themes'])):
    if detect(df_books['Themes'][i]) != 'fr' :
        url_api_transl = url + source + dest + df_books['Themes'][i]
        response = requests.get(url_api_transl)
        if response.status_code == 200:
            translated_themes = response.json().get("translation", "")
            df_books.loc[i,'Themes'] = translated_themes
        time.sleep(2)

In [78]:
#Map themes in order to clean themes in the df 
themes_mapping = {
    "Ambitions": ["Ambitions", "Luxe", "Richesse", "Pouvoir"],
    "Argent": ["Argent", "Finance", "Luxe", "Richesse"],
    "Arts et Culture": ["Musique", "Opéras", "Livret", "Partitions", "Illustrations"],
    "Biographie": ["Biographie", "Autobiographie", "Fiction biographique"],
    "Censure": ["Censure", "Critique littéraire", "Liberté d'expression", "Interdictions"],
    "Critique": ["Critique et interprétation", "Histoire et critique", "Critique", "Philosophie", "controverse", "Satire", "Histoire de la philosophie", "Histoire politique"],
    "Critique d'art": ["Critique d'art", "Art moderne", "Peinture", "Critique d'art", "Arts modernes"],
    "Coutumes": ["Coutumes", "Mœurs et coutumes"],
    "Décadence": ["Décadence", "Chute", "Esthétique", "Fiction"],
    "Drame": ["Drame", "Fiction", "Théâtre"],
    "Éducation": ["Éducation", "Éducation", "Ouvrages de jeunesse jusqu'à 1800"],
    "Éducation et Conduite de la vie": ["Éducation des filles", "Éducation des princes", "Conduite personnelle", "Éducation", "Philosophie de l'éducation"],
    "Épouvante": ["Épouvante", "Horreur", "Suspense", "Thriller"],
    "Esthétique": ["Esthétique", "Art"],
    "Fantastique": ["Fantastique", "Fiction fantastique", "Imaginaire"],
    "Fiction historique": ["Fiction historique", "Fiction", "Histoire", "Aventures", "Époque", "Événements"],
    "Héroïsme": ["Fiction", "Héroïsme", "Héroïsme"],
    "Histoire": ["Histoire", "Historique", "Historique et critique", "Société", "Évolutions sociales", "Révolution française", "Histoire", "Politique", "Révolutions"],
    "Inégalités": ["Inégalités", "Sociale"],
    "Instincts": ["Instincts", "Comportements"],
    "Intrigues": ["Fiction", "Intrigues"],
    "Justice": ["Justice", "Loi", "Droits", "Éthique"],
    "Liberté": ["Liberté", "Égalité", "Liberté religieuse", "Liberté"],
    "Cour": ["Luxe et débauche", "Château", "Fiction", "Vie de cour", "Aristocratie", "Luxe", "Critique sociale", "Roi", "Reine",  "chevalier", "Noblesse", "noblesse", "cour"],
    "Mélancolie": ["Mélancolie", "Tristesse"],
    "Mœurs": ["Mœurs et coutumes", "Vie sociale et mœurs", "Mœurs et usages"],
    "Mythes et légendes": ["Mythes", "Mythologie", "Légendes", "Fables", "Folklore"],
    "Nature": ["Nature", "Histoire naturelle", "La nature dans la littérature", "Aspects religieux de la nature"],
    "Nature et environnement": ["Nature", "Écologie", "Environnement", "Conservation", "Vie à la campagne"],
    "Oppression": ["Oppression", "Répression"],
    "Passion": ["Passion", "Émotions", "Séduction", "érotique", "sexuel", "sexe", "amour"],
    "Philosophie": ["Vie intellectuelle", "Existence", "Philosophie politique", "État", "Droit", "Liberté", "Nationalisme", "République", "Constitution", "Philosophie", "Philosophie du XVIIIe siècle", "Rationalisme", "Égalité", "Droit"],
    "Pouvoir": ["Pouvoir", "Droit", "Politique et gouvernement"],
    "Réalité et illusion": ["Réalité", "Illusion", "Perception", "L'irréel"],
    "Religion": ["Religion", "Dieu", "Christianisme", "Religieuses", "Jésuites", "Histoire de l'Église", "Église catholique", "Œuvres apologétiques", "Tolérance religieuse", "Affaire Calas", "Athéisme"],
    "Récits de guerre et combat": ["Hostilité", "Récits de guerre", "Conflits", "Histoire militaire", "chevaliers", ],
    "Société et politique": ["Etat","Jurisprudence" , "Droit", "Colonies","classes sociales", "statut", "discours et statut social", "Commerce", "Société", "Conditions sociales", "Structure sociale", "Mœurs", "Civilisation", "Culture", "Vie sociale", "État", "Science politique", "Éducation des princes", "Conduite de la vie", "Nationalisme", "Tolérance religieuse", "Conditions sociales", "Affaire","Réformateurs sociaux", "Socialisme"],
    "Théâtre": ["Théâtre", "Drame", "Théâtre en vers"],
    "Techniques": ["Technologie", "Science", "Innovations"],
    "Utopie": ["Utopie", "Littérature utopique", "Fiction spéculative", "Socialisme utopique"],
    "Voyages": ["Voyages", "Voyages imaginaires", "Voyages"],
    "Vie sociale et mœurs": ["Vie sociale", "Mœurs et coutumes", "Conditions morales", "Civilisation"],
    "Voyages et explorations": ["Voyages", "Voyages et découvertes", "Explorations"],
    "Poésie": ["Poésie", "Vers", "Chansons", "Épique", "Lyrique", "Sonnet", "Ballade"],
    "Roman": ["Romans", "Réalisme", "Roman historique", "Roman d'aventure", "Roman philosophique", "Roman social", "Roman d'amour"],
    "Fables": ["Fables", "Morales", "Allégories", "Contes moraux", "Animaux dans la littérature"],
    "Satire": ["Satire", "Humour", "Critique sociale", "Ironie", "Caricature", "Sarcasme"],
    "Conte": ["Contes", "Fiction pour enfants", "Féerie", "Contes merveilleux", "Fables", "Contes populaires"],
    "Lettres imaginaires": ["Lettres imaginaires", "Correspondance fictive", "Épistolaire", "Narration par lettres", "Philosophie épistolaire"],
    "Aventures": ["Voyages", "Animaux", "Voyages imaginaires", "mer", "naufrages" "Explorations", "Voyages réels", "Aventures géographiques", "Découvertes", "Voyages philosophiques", "Aventure et aventuriers", "Histoires d'aventure", "action et aventure"],
    "Meurtre": ["homicide", "meurtre" ],
    "Tragique": ["meurtre", "homicide", "pendaison", "tragique", "tragi-comédie", "funéraires", "mort", "victimes" ],
    "Vengeance": ["vengeance", "haine", "colère"],
    "Antiquité": ["Antiquités", "Empire romain"],
    "Comédie": ["humour", "tragi-comédie", "satire", "parodie", "comique de situation", "caricature", "absurde", "délire", "quiproquo", "rire", "buffonerie", "exagération", "jeu de mots", "Tromperie"],
    "Romantisme": ["amour", "nature", "passion", "mélancolie", "liberté", "sublime"],
    "Classicisme": ["ordre", "raison", "équilibre", "harmonie", "bienséance", "tragédie"],
    "Réalisme": ["réalité", "quotidien", "détails", "société", "observation", "critique sociale"],
    "Naturalisme": ["hérédité", "milieu", "observation scientifique", "réalisme", "mécanismes sociaux"],
    "Symbolisme": ["mystère", "suggestion", "musique", "imagination", "spiritualité"],
    "Baroque": ["exagération", "mouvement", "émotion", "illusion", "ornement"],
    "Lumières": ["raison", "progrès", "liberté", "égalité", "éducation", "philosophie"],
    "Surréalisme": ["inconscient", "rêve", "imaginaire", "liberté créative", "poésie automatique"],
    "Existentialisme": ["liberté individuelle", "absurde", "choix", "angoisse", "responsabilité"],
    "Futurisme": ["vitesse", "technologie", "modernité", "progrès", "dynamisme"],
    "Renaissance": ["humanisme", "art", "science", "antiquité", "redécouverte", "perspective"],
    "Médiéval": ["chevalerie", "foi", "mythologie", "féodalité", "croisades", "épopée"],
    "Romantisme noir": ["horreur", "mystère", "surnaturel", "gothique", "angoisse"],
    "Postmodernisme": ["fragmentation", "ironique", "pluralisme", "métafiction", "intertextualité"],
}



In [65]:
# Fonction de mapping des thèmes
def map_themes(row):
    # Diviser les thèmes par virgule
    themes = row.split(", ")
    mapped_themes = []
    
    # Vérifier et mapper chaque thème
    for theme in themes:
        # Chercher le thème dans le dictionnaire
        for key, values in themes_mapping.items():
            if theme in values:
                mapped_themes.append(key)
                break
    
    # Retourner les thèmes mappés sous forme de chaîne
    return ", ".join(mapped_themes)

# Appliquer la fonction de mapping sur la colonne "Themes"
df_books["Themes"] = df_books["Themes"].apply(map_themes)


In [67]:
# Remove empty lines of the df as iit means that themes obtained in the API where not relevant for our analysis
df_books = df_books[df_books['Themes'] != '']
df_books.reset_index(drop=True, inplace=True)


In [68]:
df_books.reset_index(drop=True, inplace=True)

In [82]:
# Remove duplicated themes
df_books['Themes'] = df_books['Themes'].apply(lambda x: set(x.split(", ")))


In [84]:
# Transform the list of themes in a string
df_books['Themes'] = df_books['Themes'].apply(lambda x: ", ".join(x))

In [87]:
df_books.to_csv('books.csv', index=False)

In [ ]:
df_books = pd.read_csv('Data/books.csv')
# TODO expliquer pourquoi CSV suffit et non un parquet

In [89]:
import s3fs

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

MY_BUCKET = "arnaudbrrt"
fs.ls(MY_BUCKET)

['arnaudbrrt/Data_libroguessr']

In [102]:
FILE_PATH_OUT_S3 = f"{MY_BUCKET}/Data_libroguessr/df_books.csv"

with fs.open(FILE_PATH_OUT_S3, "w") as file_out:
    df_books.to_csv(file_out)

In [1]:
FILE_PATH_IN_S3 = f"{MY_BUCKET}/Data_libroguessr/df_books.csv"

with fs.open(FILE_PATH_IN_S3, "r") as file_in:
    df_books = pd.read_csv(file_in)


NameError: name 'MY_BUCKET' is not defined

In [44]:
data = pd.read_csv('oeuvres_auteurs_pour_API.csv')

In [221]:
data.sample(20)

,Description,Index,Title,Author
420,"Actes et Paroles, vol. 3, by Victor Hugo ...",8454,"Actes et Paroles, vol. 3,",Victor Hugo \...
24,"Mahatma Gandhi, by Romain Rolland ...",67205,"Mahatma Gandhi,",Romain Rolland ...
208,Correspondance de Voltaire avec le roi de Prus...,25734,"Correspondance de Voltaire avec le roi de Prusse,",Francois Arouet \n Voltaire and Frederic I...
161,"Le barbier de S?lle ou la pr?ution inutile, ...",36826,"Le barbier de S?lle ou la pr?ution inutile, ...",Pierre Augustin Caron de Beaumarchais\n [Langu...
90,"Vingt mille lieues sous les mers, by Jules Ver...",54873,"Vingt mille lieues sous les mers,",Jules Verne \n [Illust...
202,"Un coeur simple, by Gustave Flaubert ...",26812,"Un coeur simple,",Gustave Flaubert ...
97,"Oeuvres complètes de Guy de Maupassant, Volume...",53247,"Oeuvres complètes de Guy de Maupassant, Volume 9,",Guy de Maupassant \n [Language: French]
96,"Honoré de Balzac, by Théophile Gautier ...",53399,"Honoré de Balzac,",Théophile Gautier ...
329,"Madame Bovary, by Gustave Flaubert ...",14155,"Madame Bovary,",Gustave Flaubert ...
72,"OEuvres complètes de Guy de Maupassant, Volume...",57430,"OEuvres complètes de Guy de Maupassant, Volume...",None


In [9]:
# Inversion de l'ordre des colonnes
df_books = df_books[['Author', 'Title']]
df_books.columns = ['Author', 'Title']
df_books

,Author,Title
0,Honoré de Balzac \n [Subtitle: La cousi...,"La Comédie humaine, Vol. 17, Études de mœurs,"
1,Guy de Maupassant \n [Language: French],Œuvres complètes de Guy de Maupassant - volume...
2,Honoré de Balzac\n [Language: French],"La Comédie humaine, Volume 16, Études philosop..."
3,Victor Hugo \n [Subti...,"Œuvres complètes de Victor Hugo,"
4,Honoré de Balzac\n [Language: French],"La Comédie humaine, Volume XV, Études philosop..."
...,...,...
471,Jules Verne \n [...,"Le Tour du Monde en 80 Jours,"
472,Jules Verne ...,"De La Terre a La Lune,"
473,Stendhal ...,"Le Rouge et le Noir,"
474,Stendhal ...,"L'Abbesse de Castro,"


In [11]:
# Pour les livres ayant deux auteurs, on remplace le "and" par une virgule
df_books['Author'] = df_books['Author'].str.replace(' and ', ', ') 
df_books

/tmp/ipykernel_22270/3118126467.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_books['Author'] = df_books['Author'].str.replace(' and ', ', ')


,Author,Title
0,Honoré de Balzac,"La Comédie humaine, Vol. 17, Études de mœurs,"
1,Guy de Maupassant,Œuvres complètes de Guy de Maupassant - volume...
2,Honoré de Balzac,"La Comédie humaine, Volume 16, Études philosop..."
3,Victor Hugo,"Œuvres complètes de Victor Hugo,"
4,Honoré de Balzac,"La Comédie humaine, Volume XV, Études philosop..."
...,...,...
471,Jules Verne,"Le Tour du Monde en 80 Jours,"
472,Jules Verne,"De La Terre a La Lune,"
473,Stendhal ...,"Le Rouge et le Noir,"
474,Stendhal ...,"L'Abbesse de Castro,"


In [12]:
# On nettoie le nom des auteurs, en enlevant les informations suivant leur nom
df_books['Author'] = df_books['Author'].str.split('\n').str[0]

# On retire les livres qui n'ont pas d'auteur associé
df_books = df_books[df_books['Author'].notna()]
df_books.reset_index(drop=True, inplace=True)

df_books

/tmp/ipykernel_22270/339208554.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_books['Author'] = df_books['Author'].str.split('\n').str[0]


,Author,Title
0,Honoré de Balzac,"La Comédie humaine, Vol. 17, Études de mœurs,"
1,Guy de Maupassant,Œuvres complètes de Guy de Maupassant - volume...
2,Honoré de Balzac,"La Comédie humaine, Volume 16, Études philosop..."
3,Victor Hugo,"Œuvres complètes de Victor Hugo,"
4,Honoré de Balzac,"La Comédie humaine, Volume XV, Études philosop..."
...,...,...
471,Jules Verne,"Le Tour du Monde en 80 Jours,"
472,Jules Verne,"De La Terre a La Lune,"
473,Stendhal ...,"Le Rouge et le Noir,"
474,Stendhal ...,"L'Abbesse de Castro,"


In [13]:
# On supprime les numéros de volume des livres losqu'il est donné
df_books['Title'] = df_books['Title'].str.replace(r'\s*,?\s*(Tome|tome|Volume|volume|Vol.|vol.|V)\s+\w+', '', regex=True)
df_books['Title'] = df_books['Title'].str.rstrip(' -') # Enlever le tirer en trop apparaissant en fin de ligne

df_books

/tmp/ipykernel_22270/831734832.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_books['Title'] = df_books['Title'].str.replace(r'\s*,?\s*(Tome|tome|Volume|volume|Vol.|vol.|V)\s+\w+', '', regex=True)
/tmp/ipykernel_22270/831734832.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_books['Title'] = df_books['Title'].str.rstrip(' -') # Enlever le tirer en trop apparaissant en fin de ligne


,Author,Title
0,Honoré de Balzac,"La Comédie humaine, Études de mœurs,"
1,Guy de Maupassant,"Œuvres complètes de Guy de Maupassant -,"
2,Honoré de Balzac,"La Comédie humaine, Études philosophiques et É..."
3,Victor Hugo,"Œuvres complètes de Victor Hugo,"
4,Honoré de Balzac,"La Comédie humaine, Études philosophiques, ..."
...,...,...
471,Jules Verne,"Le Tour du Monde en 80 Jours,"
472,Jules Verne,"De La Terre a La Lune,"
473,Stendhal ...,"Le Rouge et le Noir,"
474,Stendhal ...,"L'Abbesse de Castro,"


In [14]:
# On retire les sous-titres des livres
df_books['Title'] = df_books['Title'].str.replace(r':.*', '', regex=True)
df_books

/tmp/ipykernel_22270/3283368004.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_books['Title'] = df_books['Title'].str.replace(r':.*', '', regex=True)


,Author,Title
0,Honoré de Balzac,"La Comédie humaine, Études de mœurs,"
1,Guy de Maupassant,"Œuvres complètes de Guy de Maupassant -,"
2,Honoré de Balzac,"La Comédie humaine, Études philosophiques et É..."
3,Victor Hugo,"Œuvres complètes de Victor Hugo,"
4,Honoré de Balzac,"La Comédie humaine, Études philosophiques, ..."
...,...,...
471,Jules Verne,"Le Tour du Monde en 80 Jours,"
472,Jules Verne,"De La Terre a La Lune,"
473,Stendhal ...,"Le Rouge et le Noir,"
474,Stendhal ...,"L'Abbesse de Castro,"


In [15]:
# On enlève les virgules inutiles en fin de ligne
df_books['Title'] = df_books['Title'].str.rstrip(',')
df_books

/tmp/ipykernel_22270/1559094291.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_books['Title'] = df_books['Title'].str.rstrip(',')


,Author,Title
0,Honoré de Balzac,"La Comédie humaine, Études de mœurs"
1,Guy de Maupassant,Œuvres complètes de Guy de Maupassant -
2,Honoré de Balzac,"La Comédie humaine, Études philosophiques et É..."
3,Victor Hugo,Œuvres complètes de Victor Hugo
4,Honoré de Balzac,"La Comédie humaine, Études philosophiques, ..."
...,...,...
471,Jules Verne,Le Tour du Monde en 80 Jours
472,Jules Verne,De La Terre a La Lune
473,Stendhal ...,Le Rouge et le Noir
474,Stendhal ...,L'Abbesse de Castro


In [16]:
# On enlève les tirets inutiles en fin de ligne
df_books['Title'] = df_books['Title'].str.rstrip('-').str.rstrip()
df_books

/tmp/ipykernel_22270/1110024846.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_books['Title'] = df_books['Title'].str.rstrip('-').str.rstrip()


,Author,Title
0,Honoré de Balzac,"La Comédie humaine, Études de mœurs"
1,Guy de Maupassant,Œuvres complètes de Guy de Maupassant
2,Honoré de Balzac,"La Comédie humaine, Études philosophiques et É..."
3,Victor Hugo,Œuvres complètes de Victor Hugo
4,Honoré de Balzac,"La Comédie humaine, Études philosophiques,"
...,...,...
471,Jules Verne,Le Tour du Monde en 80 Jours
472,Jules Verne,De La Terre a La Lune
473,Stendhal ...,Le Rouge et le Noir
474,Stendhal ...,L'Abbesse de Castro


In [108]:
# Après ce nettoyage, on remarque que des livres ont des titres similaires. De ce fait, on les nettoie
df_books = remove_similar_books_from_df(df_books)
df_books

,Author,Title
0,Honoré de Balzac,"La Comédie humaine, Études de mœurs"
1,Honoré de Balzac,"La Comédie humaine, Études philosophiques et É..."
2,Victor Hugo,Œuvres complètes de Victor Hugo
3,Honoré de Balzac,"La Comédie humaine, Études philosophiques,"
4,Honoré de Balzac,La Comédie humaine
...,...,...
380,Jules Verne,Le Tour du Monde en 80 Jours
381,Jules Verne,De La Terre a La Lune
382,Stendhal ...,Le Rouge et le Noir
383,Stendhal ...,L'Abbesse de Castro


In [110]:
df_books.sample(20)

,Author,Title
351,Anatole France ...,Le Jardin d'Epicure
313,Guy de Maupassant ...,Monsieur Parent
384,Stendhal,La Chartreuse de Parme
101,Octave Mirbeau ...,Le Calvaire
7,None,Œuvres complètes de Guy de Maupassant ...
154,Maurice Clouard,Alfred de Musset et George Sand
383,Stendhal ...,L'Abbesse de Castro
32,Louis Fagan and Prosper Mérimée,Lettres à M. Panizzi
141,Eugène Sue,"Atar-Gull, Un Corsaire, Le Parisien en Mer, Vo..."
83,Denis Diderot,Lettres à Mademoiselle de Volland


In [17]:

def get_themes(author, title):
    """
    This function fetches information about a book by a given author from the Open Library API.
    It returns the themes (subjects) associated with the book that has the largest number of subjects among the results.

    Arguments:
        - author (str): The name of the book's author.
        - title (str): The title of the book.

    Workflow:
        1. Constructs a query to the Open Library API using the author's name and book's title.
        2. Retrieves JSON results containing information about books matching the query.
        3. Filters books to keep only those that have defined themes ("subject").
        4. Selects the book with the largest number of themes from the filtered results.
        5. Returns the selected book's information: author, title, year of publication, and themes.

    Returns:
        - A list containing a single tuple in the format:
          [(author, title, year, themes)]
          where:
            - author (str): The author's name.
            - title (str): The book's title.
            - year (str): The book's first publication year (if available).
            - themes (str): The book's associated themes, joined into a string separated by commas.
          If no book with themes is found, it returns a list with empty values:
          [(author, title, '', '')].
    """
    
    url_api = "https://openlibrary.org/search.json?"

    # Build the API URL with query parameters for author and title
    url = url_api + f'q=author:{author}&fields={title},first_publish_year,subject'
    req = requests.get(url)  # Send the HTTP request
    time.sleep(2)  # Pause for 2 seconds to avoid overloading the API
    
    # Check if the request was successful (HTTP status code 200)
    if req.status_code == 200:
        books = req.json().get("docs", [])  # Retrieve the list of books from the JSON response
        
        # Filter books that contain the "subject" field (themes)
        books_with_themes = [
            book for book in books
            if book.get("subject")  # Keep only books with defined themes
        ]
        
        if books_with_themes:  # If books with themes are found
            # Select the book with the largest number of themes
            best_book = max(books_with_themes, key=lambda book: len(book["subject"]))
            
            # Retrieve the themes of the selected book and join them into a single string
            themes = ", ".join(best_book.get("subject", ''))  
            
            # Retrieve the year of first publication
            year = best_book.get("first_publish_year", '')  
            
            # Return the book information as a list of a single tuple
            return [(author, title, year, themes)] 
        else:
            # If no book with themes is found, return empty values
            return [(author, title, '', '')]
    else:
        # If the request fails, return empty values
        return [(author, title, '', '')]

def get_books_info(authors, titles):
    """
    This function takes two lists as input: a list of authors and a list of titles.
    It calls the `get_themes` function for each author-title combination and returns a flattened list of tuples containing the book information.

    Arguments:
        - authors (list): List of authors' names.
        - titles (list): List of book titles.

    Returns:
        - A DataFrame containing columns: 'Author', 'Title', 'Year', 'Themes' with the book information for each author-title combination.
    """
    list_books = []

    # For each author and title, retrieve the book information
    for author, title in zip(authors, titles):
        list_books.append(get_themes(author, title))

    # Flatten the list of lists of tuples into a single list
    flattened_books = [book for author_data in list_books for book in author_data]

    return pd.DataFrame(flattened_books, columns=['Author', 'Title', 'Year', 'Themes'])


def map_themes(row):
    """
    Maps individual themes from a comma-separated input string to their corresponding categories based on 
    the `themes_mapping` dictionary. The function splits the input string of themes, searches for each theme 
    in the dictionary, and returns a string of category names.

    Parameters:
    row (str): A string of themes, separated by commas and spaces. Each theme corresponds to one or more 
               categories in the `themes_mapping` dictionary.

    Returns:
    str: A comma-separated string containing the categories (keys) from the `themes_mapping` dictionary 
         that correspond to the input themes. If a theme is found in multiple categories, only the first 
         matching category is returned for that theme.

    """
    
    # Split the input string by comma and space (", ") into individual themes
    themes = row.split(", ")
    
    # Initialize an empty list to store the mapped themes (categories)
    mapped_themes = []  
    
    # Iterate through each theme in the list
    for theme in themes:
        # Loop through each key-value pair in the themes_mapping dictionary
        for key, values in themes_mapping.items():
            # If the theme is found in the values (list) for the current key
            if theme in values:
                # Add the corresponding category (key) to the list of mapped themes
                mapped_themes.append(key)  
                break  # Exit the inner loop once a match is found
    
    # Return the mapped themes as a comma-separated string
    return ", ".join(mapped_themes)

In [24]:
# Convertir en DataFrame
df_books = get_books_info(df_books['Author'], df_books['Title'])

df_books

,Author,Title,Year,Themes
0,Honoré de Balzac,"La Comédie humaine, Études de mœurs",1800,"Fiction, France in fiction, Older men in ficti..."
1,Guy de Maupassant,Œuvres complètes de Guy de Maupassant,1893,"Translations from French, Fiction in English, ..."
2,Honoré de Balzac,"La Comédie humaine, Études philosophiques et É...",1800,"Fiction, France in fiction, Older men in ficti..."
3,Victor Hugo,Œuvres complètes de Victor Hugo,1800,"Poor, Fiction, Readers, Fiction, historical, F..."
4,Honoré de Balzac,"La Comédie humaine, Études philosophiques,",,
...,...,...,...,...
471,Jules Verne,Le Tour du Monde en 80 Jours,1872,"Viajes alrededor del mundo, Translations into ..."
472,Jules Verne,De La Terre a La Lune,1872,"Viajes alrededor del mundo, Translations into ..."
473,Stendhal ...,Le Rouge et le Noir,1800,"Fiction, Young men, Social life and customs, Y..."
474,Stendhal ...,L'Abbesse de Castro,,


In [114]:
# delete lines of the DF for which themes were found
df_books = df_books[df_books['Themes'] != '']
df_books.reset_index(drop=True, inplace=True)

Comme cette API fournit les thèmes en anglais, on utilise une API qui permet de traduire les textes : l'API Lingva qui est gratuite et qui ne requiert pas de token.

In [116]:
url = "https://lingva.ml/api/v1/"
source = 'auto/'
dest = 'fr/'

for i in range(len(df_books['Themes'])):
    if detect(df_books['Themes'][i]) != 'fr' :
        url_api_transl = url + source + dest + df_books['Themes'][i]
        response = requests.get(url_api_transl)
        if response.status_code == 200:
            translated_themes = response.json().get("translation", "")
            df_books.loc[i,'Themes'] = translated_themes
        time.sleep(2)

/tmp/ipykernel_6929/4052680809.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_books.loc[i,'Themes'] = translated_themes


In [192]:
#TODO : enlever
"""df_books.to_csv('mapping', index=False)
df_books = pd.read_csv('mapping')
df_books"""

,Author,Title,Year,Themes
0,Honoré de Balzac,"La Comédie humaine, Études de mœurs",1800,"Fiction, France in fiction, Older men in ficti..."
1,Honoré de Balzac,"La Comédie humaine, Études philosophiques et É...",1800,"Fiction, France in fiction, Older men in ficti..."
2,Victor Hugo,Œuvres complètes de Victor Hugo,1800,"Poor, Fiction, Readers, Fiction, historical, F..."
3,Honoré de Balzac,"La Comédie humaine, Études philosophiques,",1800,"Fiction, France in fiction, Older men in ficti..."
4,Honoré de Balzac,La Comédie humaine,1800,"Fiction, France in fiction, Older men in ficti..."
...,...,...,...,...
295,Stendhal ...,Les Cenci,1800,"Fiction, Young men, Social life and customs, Y..."
296,Jules Verne,Le Tour du Monde en 80 Jours,1872,"Voyages autour du monde, Traductions en gujara..."
297,Jules Verne,De La Terre a La Lune,1872,"Voyages autour du monde, Traductions en gujara..."
298,Stendhal ...,Le Rouge et le Noir,1800,"Fiction, Young men, Social life and customs, Y..."


Une fois que les thèmes sont traduits, on les nettoie pour ne garder que les thèmes pertinents et les regrouper dans des catégories plus générales. Pour cela, on réalise un _mapping_ qui regroupe, dans un thème, une liste des mots faisant référence à ce thème. Ce _mapping_ est réalisé manuellement. Cela permet également de nettoyer les thèmes qui ne sont pas pertinents, ce qui explique pourquoi certains livres n'auront plus de thèmes associés.

In [193]:
themes_mapping = {
    "Ambitions": ["Ambitions", "Luxe", "Pouvoir", "Richesse"],
    "Antiquité": ["Antiquités", "Empire romain"],
    "Argent": ["Argent", "Finance", "Luxe", "Richesse"],
    "Arts et Culture": ["Illustrations", "Livret", "Musique", "Opéras", "Partitions"],
    "Aventures": ["Animaux", "Découvertes", "Explorations", "Histoires d'aventure", "mer", "naufrages", "Voyages", "Voyages philosophiques", "Voyages imaginaires"],
    "Baroque": ["Émotion", "Exagération", "Illusion", "Mouvement", "Ornement"],
    "Biographie": ["Autobiographie", "Biographie", "Fiction biographique"],
    "Censure": ["Censure", "Critique littéraire", "Interdictions", "Liberté d'expression"],
    "Comédie": ["absurde", "caricature", "comique de situation", "délire", "exagération", "humour", "parodie", "quiproquo", "rire", "satire", "Tromperie"],
    "Critique": ["Critique", "Critique et interprétation", "Histoire de la philosophie", "Histoire politique", "Philosophie", "Satire", "controverse", "Histoire et critique"],
    "Critique d'art": ["Art moderne", "Critique d'art", "Critique d'art", "Peinture", "Arts modernes"],
    "Coutumes": ["Coutumes", "Mœurs et coutumes"],
    "Décadence": ["Chute", "Décadence", "Esthétique", "Fiction"],
    "Drame": ["Drame", "Fiction", "Théâtre"],
    "Éducation": ["Éducation", "Ouvrages de jeunesse jusqu'à 1800"],
    "Éducation et Conduite de la vie": ["Conduite personnelle", "Éducation", "Éducation des filles", "Éducation des princes", "Philosophie de l'éducation"],
    "Épouvante": ["Épouvante", "Horreur", "Suspense", "Thriller"],
    "Esthétique": ["Art", "Esthétique"],
    "Existentialisme": ["Angoisse", "Absurde", "Choix", "Liberté individuelle", "Responsabilité"],
    "Fantastique": ["Fantastique", "Fiction fantastique", "Imaginaire"],
    "Fables": ["Allégories", "Animaux dans la littérature", "Contes moraux", "Fables", "Morales"],
    "Fiction historique": ["Aventures", "Époque", "Événements", "Fiction", "Fiction historique", "Histoire"],
    "Héroïsme": ["Fiction", "Héroïsme"],
    "Histoire": ["Évolutions sociales", "Histoire", "Historique", "Historique et critique", "Politique", "Révolution française", "Révolutions", "Société"],
    "Inégalités": ["Inégalités", "Sociale"],
    "Instincts": ["Comportements", "Instincts"],
    "Intrigues": ["Fiction", "Intrigues"],
    "Justice": ["Droits", "Éthique", "Justice", "Loi"],
    "Liberté": ["Égalité", "Liberté", "Liberté religieuse"],
    "Lumières": ["Égalité", "Liberté", "Philosophie", "Progrès", "Raison", "Éducation"],
    "Médiéval": ["Chevalerie", "Épopée", "Féodalité", "Foi", "Croisades", "Mythologie"],
    "Mélancolie": ["Mélancolie", "Tristesse"],
    "Mœurs": ["Mœurs et coutumes", "Mœurs et usages", "Vie sociale et mœurs"],
    "Mythes et légendes": ["Fables", "Légendes", "Mythes", "Mythologie", "Folklore"],
    "Nature": ["Aspects religieux de la nature", "Histoire naturelle", "La nature dans la littérature", "Nature"],
    "Nature et environnement": ["Conservation", "Environnement", "Écologie", "Nature", "Vie à la campagne"],
    "Oppression": ["Oppression", "Répression"],
    "Passion": ["Amour", "Émotions", "Passion", "Séduction", "érotique", "sexuel", "sexe"],
    "Philosophie": ["Constitution", "Droit", "Égalité", "Existence", "Liberté", "Nationalisme", "Philosophie", "Philosophie du XVIIIe siècle", "Philosophie politique", "Rationalisme", "République", "Vie intellectuelle"],
     "Poésie": ["Ballade", "Chansons", "Épique", "Lyrique", "Poésie", "Sonnet", "Vers"],
    "Postmodernisme": ["Fragmentation", "Intertextualité", "Ironique", "Métafiction", "Pluralisme"],
    "Pouvoir": ["Droit", "Politique et gouvernement", "Pouvoir"],
    "Réalité et illusion": ["Illusion", "Perception", "Réalité", "L'irréel"],
    "Religion": ["Athéisme", "Christianisme", "Dieu", "Église catholique", "Histoire de l'Église", "Religieuses", "Tolérance religieuse", "Affaire Calas"],
    "Récits de guerre et combat": ["Conflits", "Hostilité", "Récits de guerre", "chevaliers", "Histoire militaire"],
    "Renaissance": ["Antiquité", "Art", "Humanisme", "Perspective", "Redécouverte", "Science"],
    "Roman": ["Réalisme", "Roman d'aventure", "Roman d'amour", "Roman philosophique", "Roman historique", "Roman social"],
    "Romantisme": ["amour", "liberté", "mélancolie", "nature", "passion", "sublime"],
    "Satire": ["Caricature", "Critique sociale", "Humour", "Ironie", "Sarcasme", "Satire"],
    "Société et politique": ["Affaire", "Classes sociales", "Commerce", "Conditions sociales", "Culture", "Discours et statut social", "État", "Éducation des princes", "Jurisprudence", "Mœurs", "Nationalisme", "Réformateurs sociaux", "Science politique", "Société", "Socialisme", "Tolérance religieuse"],
    "Surréalisme": ["Imaginaire", "Liberté créative", "Poésie automatique", "Rêve", "Inconscient"],
    "Techniques": ["Innovations", "Science", "Technologie"],
    "Théâtre": ["Drame", "Théâtre", "Théâtre en vers"],
    "Tragique": ["funéraires", "homicide", "meurtre", "pendaison", "tragique", "victimes", "tragi-comédie", "mort"],
    "Utopie": ["Fiction spéculative", "Littérature utopique", "Socialisme utopique", "Utopie"],
    "Vengeance": ["colère", "haine", "vengeance"],
    "Vie sociale et mœurs": ["Conditions morales", "Civilisation", "Mœurs et coutumes", "Vie sociale"],
    "Voyages": ["Voyages", "Voyages imaginaires"],
    "Voyages et explorations": ["Découvertes", "Explorations", "Voyages et découvertes"],
}



In [194]:
# Appliquer la fonction de mapping sur la colonne "Themes"
df_books["Themes"] = df_books["Themes"].apply(map_themes)
df_books


,Author,Title,Year,Themes
0,Honoré de Balzac,"La Comédie humaine, Études de mœurs",1800,"Décadence, Décadence, Décadence, Décadence, Dé..."
1,Honoré de Balzac,"La Comédie humaine, Études philosophiques et É...",1800,"Décadence, Décadence, Décadence, Décadence, Dé..."
2,Victor Hugo,Œuvres complètes de Victor Hugo,1800,"Décadence, Décadence, Décadence, Décadence, Dé..."
3,Honoré de Balzac,"La Comédie humaine, Études philosophiques,",1800,"Décadence, Décadence, Décadence, Décadence, Dé..."
4,Honoré de Balzac,La Comédie humaine,1800,"Décadence, Décadence, Décadence, Décadence, Dé..."
...,...,...,...,...
295,Stendhal ...,Les Cenci,1800,"Décadence, Critique, Décadence, Roman, Roman, ..."
296,Jules Verne,Le Tour du Monde en 80 Jours,1872,"Décadence, Fiction historique, Aventures, Aven..."
297,Jules Verne,De La Terre a La Lune,1872,"Décadence, Fiction historique, Aventures, Aven..."
298,Stendhal ...,Le Rouge et le Noir,1800,"Décadence, Critique, Décadence, Roman, Roman, ..."


In [195]:
# On enlève les livres qui n'ont pas de thèmes pertinents associés
df_books = df_books[df_books['Themes'] != '']
df_books.reset_index(drop=True, inplace=True)
df_books

,Author,Title,Year,Themes
0,Honoré de Balzac,"La Comédie humaine, Études de mœurs",1800,"Décadence, Décadence, Décadence, Décadence, Dé..."
1,Honoré de Balzac,"La Comédie humaine, Études philosophiques et É...",1800,"Décadence, Décadence, Décadence, Décadence, Dé..."
2,Victor Hugo,Œuvres complètes de Victor Hugo,1800,"Décadence, Décadence, Décadence, Décadence, Dé..."
3,Honoré de Balzac,"La Comédie humaine, Études philosophiques,",1800,"Décadence, Décadence, Décadence, Décadence, Dé..."
4,Honoré de Balzac,La Comédie humaine,1800,"Décadence, Décadence, Décadence, Décadence, Dé..."
...,...,...,...,...
271,Stendhal ...,Les Cenci,1800,"Décadence, Critique, Décadence, Roman, Roman, ..."
272,Jules Verne,Le Tour du Monde en 80 Jours,1872,"Décadence, Fiction historique, Aventures, Aven..."
273,Jules Verne,De La Terre a La Lune,1872,"Décadence, Fiction historique, Aventures, Aven..."
274,Stendhal ...,Le Rouge et le Noir,1800,"Décadence, Critique, Décadence, Roman, Roman, ..."


In [196]:
# On retire les thèmes en double et les auteurs 
df_books['Themes'] = df_books['Themes'].apply(lambda x: ', '.join(sorted(set(x.split(', ')), key=x.split(', ').index)))
df_books = df_books[df_books['Author'].notna()]
df_books = df_books[df_books['Themes'] != '']
df_books.reset_index(drop=True, inplace=True)

df_books


/tmp/ipykernel_6929/2851936720.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_books['Themes'] = df_books['Themes'].apply(lambda x: ', '.join(sorted(set(x.split(', ')), key=x.split(', ').index)))


,Author,Title,Year,Themes
0,Honoré de Balzac,"La Comédie humaine, Études de mœurs",1800,Décadence
1,Honoré de Balzac,"La Comédie humaine, Études philosophiques et É...",1800,Décadence
2,Victor Hugo,Œuvres complètes de Victor Hugo,1800,"Décadence, Roman, Coutumes"
3,Honoré de Balzac,"La Comédie humaine, Études philosophiques,",1800,Décadence
4,Honoré de Balzac,La Comédie humaine,1800,Décadence
...,...,...,...,...
271,Stendhal ...,Les Cenci,1800,"Décadence, Critique, Roman, Coutumes"
272,Jules Verne,Le Tour du Monde en 80 Jours,1872,"Décadence, Fiction historique, Aventures, Tech..."
273,Jules Verne,De La Terre a La Lune,1872,"Décadence, Fiction historique, Aventures, Tech..."
274,Stendhal ...,Le Rouge et le Noir,1800,"Décadence, Critique, Roman, Coutumes"


In [200]:
df_books.to_csv('mapping1', index=False)

In [201]:
# Après le nettoyage, le data frame prend la forme suivante :
df_books = pd.read_csv('mapping1')
df_books

,Author,Title,Year,Themes
0,Honoré de Balzac,"La Comédie humaine, Études de mœurs",1800,Décadence
1,Honoré de Balzac,"La Comédie humaine, Études philosophiques et É...",1800,Décadence
2,Victor Hugo,Œuvres complètes de Victor Hugo,1800,"Décadence, Roman, Coutumes"
3,Honoré de Balzac,"La Comédie humaine, Études philosophiques,",1800,Décadence
4,Honoré de Balzac,La Comédie humaine,1800,Décadence
...,...,...,...,...
271,Stendhal ...,Les Cenci,1800,"Décadence, Critique, Roman, Coutumes"
272,Jules Verne,Le Tour du Monde en 80 Jours,1872,"Décadence, Fiction historique, Aventures, Tech..."
273,Jules Verne,De La Terre a La Lune,1872,"Décadence, Fiction historique, Aventures, Tech..."
274,Stendhal ...,Le Rouge et le Noir,1800,"Décadence, Critique, Roman, Coutumes"


On poursuit le nettoyage de la base de données.

In [202]:
df_books['Author_Cleaned'] = df_books['Author'].apply(lambda x: x.rstrip())
# Comme La Comédie humaine est l'oeuvre de Balzac est non pas un livre, on le retire de la base de données
df_books = df_books[df_books['Author_Cleaned'] != "Honoré de Balzac"]
df_books.reset_index(drop=True, inplace=True)
df_books

,Author,Title,Year,Themes,Author_Cleaned
0,Honoré de Balzac,"La Comédie humaine, Études de mœurs",1800,Décadence,Honoré de Balzac
1,Honoré de Balzac,"La Comédie humaine, Études philosophiques et É...",1800,Décadence,Honoré de Balzac
2,Victor Hugo,Œuvres complètes de Victor Hugo,1800,"Décadence, Roman, Coutumes",Victor Hugo
3,Honoré de Balzac,"La Comédie humaine, Études philosophiques,",1800,Décadence,Honoré de Balzac
4,Honoré de Balzac,La Comédie humaine,1800,Décadence,Honoré de Balzac
...,...,...,...,...,...
271,Stendhal ...,Les Cenci,1800,"Décadence, Critique, Roman, Coutumes",Stendhal
272,Jules Verne,Le Tour du Monde en 80 Jours,1872,"Décadence, Fiction historique, Aventures, Tech...",Jules Verne
273,Jules Verne,De La Terre a La Lune,1872,"Décadence, Fiction historique, Aventures, Tech...",Jules Verne
274,Stendhal ...,Le Rouge et le Noir,1800,"Décadence, Critique, Roman, Coutumes",Stendhal


In [205]:
df_books['Author_Cleaned'] = df_books['Author_Cleaned'].apply(lambda x: x.rstrip())
df_books

/tmp/ipykernel_6929/1289416502.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_books['Author_Cleaned'] = df_books['Author_Cleaned'].apply(lambda x: x.rstrip())


,Author,Title,Year,Themes,Author_Cleaned
0,Victor Hugo,Œuvres complètes de Victor Hugo,1800,"Décadence, Roman, Coutumes",Victor Hugo
1,Alexandre Dumas ...,Pauline,1830,Décadence,Alexandre Dumas
2,Guy de Maupassant,Œuvres complètes de Guy de Maupassant,1893,"Décadence, Pouvoir, Société et politique, Ambi...",Guy de Maupassant
3,Théophile Gautier ...,Constantinople,1834,Décadence,Théophile Gautier
4,Romain Rolland ...,Clerambault,1908,"Critique, Biographie, Arts et Culture",Romain Rolland
...,...,...,...,...,...
263,Stendhal ...,Les Cenci,1800,"Décadence, Critique, Roman, Coutumes",Stendhal
264,Jules Verne,Le Tour du Monde en 80 Jours,1872,"Décadence, Fiction historique, Aventures, Tech...",Jules Verne
265,Jules Verne,De La Terre a La Lune,1872,"Décadence, Fiction historique, Aventures, Tech...",Jules Verne
266,Stendhal ...,Le Rouge et le Noir,1800,"Décadence, Critique, Roman, Coutumes",Stendhal


In [273]:
# Supprimer les lignes où le titre commence par 'Œuvres complètes'
df_books = df_books[~df_books['Title'].str.startswith(('Œuvres complètes', 'Œuvres', 'Oeuvres', 'Poésies Complètes', 'Ouvres'))]

# Réinitialiser l'index du DataFrame
df_books.reset_index(drop=True, inplace=True)
df_books

,Author,Title,Year,Themes,Author_Cleaned
0,Alexandre Dumas ...,Pauline,1830,Décadence,Alexandre Dumas
1,Alexandre Dumas,Les trois mousquetaires,1830,Décadence,Alexandre Dumas
2,Alexandre Dumas ...,Gabriel Lambert,1830,Décadence,Alexandre Dumas
3,Alexandre Dumas ...,Le comte de Moret,1830,Décadence,Alexandre Dumas
4,Alexandre Dumas ...,La tulipe noire,1830,Décadence,Alexandre Dumas
...,...,...,...,...,...
91,Émile Zola ...,La curée,1885,"Décadence, Société et politique",Émile Zola
92,Émile Zola ...,Le rêve,1885,"Décadence, Société et politique",Émile Zola
93,Émile Zola,Au bonheur des dames,1885,"Décadence, Société et politique",Émile Zola
94,Émile Zola ...,Contes à Ninon,1885,"Décadence, Société et politique",Émile Zola


In [207]:
df_books = df_books[df_books['Year'] < 1900]
df_books.reset_index(drop=True, inplace=True)
df_books


,Author,Title,Year,Themes,Author_Cleaned
0,Alexandre Dumas ...,Pauline,1830,Décadence,Alexandre Dumas
1,Théophile Gautier ...,Constantinople,1834,Décadence,Théophile Gautier
2,George Sand ...,Les dames vertes,1846,Roman,George Sand
3,Jean-Jacques Rousseau,Les Rêveries du Promeneur Solitaire,1762,"Société et politique, Décadence, Critique, Lib...",Jean-Jacques Rousseau
4,Edmond Rostand,Deux romanciers de Provence,1821,"Drame, Fiction historique, Décadence, Arts et ...",Edmond Rostand
...,...,...,...,...,...
217,Stendhal ...,Les Cenci,1800,"Décadence, Critique, Roman, Coutumes",Stendhal
218,Jules Verne,Le Tour du Monde en 80 Jours,1872,"Décadence, Fiction historique, Aventures, Tech...",Jules Verne
219,Jules Verne,De La Terre a La Lune,1872,"Décadence, Fiction historique, Aventures, Tech...",Jules Verne
220,Stendhal ...,Le Rouge et le Noir,1800,"Décadence, Critique, Roman, Coutumes",Stendhal


In [208]:
df_books = remove_similar_books_from_df(df_books)
df_books

,Author,Title,Year,Themes,Author_Cleaned
0,Alexandre Dumas ...,Pauline,1830,Décadence,Alexandre Dumas
1,Théophile Gautier ...,Constantinople,1834,Décadence,Théophile Gautier
2,George Sand ...,Les dames vertes,1846,Roman,George Sand
3,Jean-Jacques Rousseau,Les Rêveries du Promeneur Solitaire,1762,"Société et politique, Décadence, Critique, Lib...",Jean-Jacques Rousseau
4,Edmond Rostand,Deux romanciers de Provence,1821,"Drame, Fiction historique, Décadence, Arts et ...",Edmond Rostand
...,...,...,...,...,...
217,Stendhal ...,Les Cenci,1800,"Décadence, Critique, Roman, Coutumes",Stendhal
218,Jules Verne,Le Tour du Monde en 80 Jours,1872,"Décadence, Fiction historique, Aventures, Tech...",Jules Verne
219,Jules Verne,De La Terre a La Lune,1872,"Décadence, Fiction historique, Aventures, Tech...",Jules Verne
220,Stendhal ...,Le Rouge et le Noir,1800,"Décadence, Critique, Roman, Coutumes",Stendhal


In [209]:
df_books.to_csv('books_a_cleaner', index = False)

In [212]:
df_books = pd.read_csv('books_a_cleaner')
df_books

,Author,Title,Year,Themes,Author_Cleaned
0,Alexandre Dumas ...,Pauline,1830,Décadence,Alexandre Dumas
1,Théophile Gautier ...,Constantinople,1834,Décadence,Théophile Gautier
2,George Sand ...,Les dames vertes,1846,Roman,George Sand
3,Jean-Jacques Rousseau,Les Rêveries du Promeneur Solitaire,1762,"Société et politique, Décadence, Critique, Lib...",Jean-Jacques Rousseau
4,Edmond Rostand,Deux romanciers de Provence,1821,"Drame, Fiction historique, Décadence, Arts et ...",Edmond Rostand
...,...,...,...,...,...
217,Stendhal ...,Les Cenci,1800,"Décadence, Critique, Roman, Coutumes",Stendhal
218,Jules Verne,Le Tour du Monde en 80 Jours,1872,"Décadence, Fiction historique, Aventures, Tech...",Jules Verne
219,Jules Verne,De La Terre a La Lune,1872,"Décadence, Fiction historique, Aventures, Tech...",Jules Verne
220,Stendhal ...,Le Rouge et le Noir,1800,"Décadence, Critique, Roman, Coutumes",Stendhal


In [213]:
# Étape 1: Compter le nombre de livres par auteur
author_counts = df_books['Author_Cleaned'].value_counts()
print(author_counts) ####### TODO on remarque que Sand a 48 livres dans le DF soit biais 
# Étape 2: Filtrer les auteurs ayant plus de 2 livres
authors_with_more_than_two_books = author_counts[author_counts > 2].index

# Étape 3: Filtrer et garder les 2 premiers livres par auteur
df_books = (
    df_books[df_books['Author_Cleaned'].isin(authors_with_more_than_two_books)]
    .sort_values(['Author_Cleaned', 'Year'])  # Trier par auteur et par année ou autre critère
    .groupby('Author_Cleaned')
    .head(10)  # Garder les 10 premiers par groupe
)
df_books.reset_index(drop=True, inplace=True)

df_books

Author_Cleaned
George Sand                                        48
Jules Verne                                        34
Alexandre Dumas                                    29
Victor Hugo                                        15
Guy de Maupassant                                  15
Voltaire                                           10
Émile Zola                                          9
Alphonse Daudet                                     9
Gustave Flaubert                                    7
Théophile Gautier                                   7
Stendhal                                            5
Edmond de Goncourt, Jules de Goncourt               4
Alexandre Dumas, Pere                               4
Eugène Sue                                          3
Edmond de Goncourt                                  3
Jean de La Fontaine                                 3
Paul Scarron                                        2
Octave Mirbeau                                      2
Edmond Rostan

,Author,Title,Year,Themes,Author_Cleaned
0,Alexandre Dumas ...,Pauline,1830,Décadence,Alexandre Dumas
1,Alexandre Dumas,Les trois mousquetaires of 2,1830,Décadence,Alexandre Dumas
2,Alexandre Dumas,Les trois mousquetaires of 2,1830,Décadence,Alexandre Dumas
3,Alexandre Dumas ...,Gabriel Lambert,1830,Décadence,Alexandre Dumas
4,Alexandre Dumas ...,Le comte de Moret,1830,Décadence,Alexandre Dumas
...,...,...,...,...,...
109,Émile Zola ...,Le rêve,1885,"Décadence, Société et politique",Émile Zola
110,Émile Zola,Au bonheur des dames,1885,"Décadence, Société et politique",Émile Zola
111,Émile Zola,Nouveaux Contes à Ninon,1885,"Décadence, Société et politique",Émile Zola
112,Émile Zola ...,Contes à Ninon,1885,"Décadence, Société et politique",Émile Zola


In [189]:
df_books.sample(20)

,Author,Title,Year,Themes
45,Guy de Maupassant,Contes du jour et de la nuit,1893,"Décadence, Pouvoir, Société et politique, Ambi..."
44,Guy de Maupassant,La vie errante,1893,"Décadence, Pouvoir, Société et politique, Ambi..."
95,Émile Zola,Le rêve,1885,"Décadence, Société et politique"
65,Stendhal,La Chartreuse de Parme,1800,"Décadence, Critique, Roman, Coutumes"
0,Alexandre Dumas,Pauline,1830,Décadence
22,"Edmond de Goncourt, Jules de Goncourt","Quelques cr?ures de ce temps,",1862,"Mœurs, Fiction historique, Décadence, Coutumes..."
25,George Sand,Mauprat,1846,Roman
76,Victor Hugo,Le Rhin. T. IV,1800,"Décadence, Roman, Coutumes"
88,Voltaire,Memnon,1746,"Critique, Décadence, Société et politique, Vie..."
52,Jules Verne,Les grands navigateurs du XIIIe siècle,1872,"Décadence, Fiction historique, Aventures, Tech..."


In [250]:
df_books['Title'] = df_books['Title'].apply(lambda x: x.split('.')[0])

df_books

,Author,Title,Year,Themes,Author_Cleaned,Title_Cleaned
0,Alexandre Dumas ...,Pauline,1830,Décadence,Alexandre Dumas,Pauline
1,Alexandre Dumas,Les trois mousquetaires of 2,1830,Décadence,Alexandre Dumas,Les trois mousquetaires of 2
2,Alexandre Dumas,Les trois mousquetaires of 2,1830,Décadence,Alexandre Dumas,Les trois mousquetaires of 2
3,Alexandre Dumas ...,Gabriel Lambert,1830,Décadence,Alexandre Dumas,Gabriel Lambert
4,Alexandre Dumas ...,Le comte de Moret,1830,Décadence,Alexandre Dumas,Le comte de Moret
...,...,...,...,...,...,...
109,Émile Zola ...,Le rêve,1885,"Décadence, Société et politique",Émile Zola,Le rêve
110,Émile Zola,Au bonheur des dames,1885,"Décadence, Société et politique",Émile Zola,Au bonheur des dames
111,Émile Zola,Nouveaux Contes à Ninon,1885,"Décadence, Société et politique",Émile Zola,Nouveaux Contes à Ninon
112,Émile Zola ...,Contes à Ninon,1885,"Décadence, Société et politique",Émile Zola,Contes à Ninon


In [259]:
df_books = df_books.drop_duplicates(subset='Title')
df_books

,Author,Title,Year,Themes,Author_Cleaned
0,Alexandre Dumas ...,Pauline,1830,Décadence,Alexandre Dumas
1,Alexandre Dumas,Les trois mousquetaires of 2,1830,Décadence,Alexandre Dumas
3,Alexandre Dumas ...,Gabriel Lambert,1830,Décadence,Alexandre Dumas
4,Alexandre Dumas ...,Le comte de Moret,1830,Décadence,Alexandre Dumas
5,Alexandre Dumas ...,La tulipe noire,1830,Décadence,Alexandre Dumas
...,...,...,...,...,...
109,Émile Zola ...,Le rêve,1885,"Décadence, Société et politique",Émile Zola
110,Émile Zola,Au bonheur des dames,1885,"Décadence, Société et politique",Émile Zola
111,Émile Zola,Nouveaux Contes à Ninon,1885,"Décadence, Société et politique",Émile Zola
112,Émile Zola ...,Contes à Ninon,1885,"Décadence, Société et politique",Émile Zola


In [261]:
df_books = df_books[df_books['Title'] != 'Le Rhin I']


In [266]:
df_books = df_books[~df_books['Title'].str.contains('\?', regex=True)]
df_books

<>:1: SyntaxWarning: invalid escape sequence '\?'
<>:1: SyntaxWarning: invalid escape sequence '\?'
/tmp/ipykernel_6929/39831067.py:1: SyntaxWarning: invalid escape sequence '\?'
  df_books = df_books[~df_books['Title'].str.contains('\?', regex=True)]


,Author,Title,Year,Themes,Author_Cleaned
0,Alexandre Dumas ...,Pauline,1830,Décadence,Alexandre Dumas
1,Alexandre Dumas,Les trois mousquetaires of 2,1830,Décadence,Alexandre Dumas
3,Alexandre Dumas ...,Gabriel Lambert,1830,Décadence,Alexandre Dumas
4,Alexandre Dumas ...,Le comte de Moret,1830,Décadence,Alexandre Dumas
5,Alexandre Dumas ...,La tulipe noire,1830,Décadence,Alexandre Dumas
...,...,...,...,...,...
109,Émile Zola ...,Le rêve,1885,"Décadence, Société et politique",Émile Zola
110,Émile Zola,Au bonheur des dames,1885,"Décadence, Société et politique",Émile Zola
111,Émile Zola,Nouveaux Contes à Ninon,1885,"Décadence, Société et politique",Émile Zola
112,Émile Zola ...,Contes à Ninon,1885,"Décadence, Société et politique",Émile Zola


In [267]:
df_books['Title'] = df_books['Title'].apply(lambda x: x.split(' of')[0])
df_books

,Author,Title,Year,Themes,Author_Cleaned
0,Alexandre Dumas ...,Pauline,1830,Décadence,Alexandre Dumas
1,Alexandre Dumas,Les trois mousquetaires,1830,Décadence,Alexandre Dumas
3,Alexandre Dumas ...,Gabriel Lambert,1830,Décadence,Alexandre Dumas
4,Alexandre Dumas ...,Le comte de Moret,1830,Décadence,Alexandre Dumas
5,Alexandre Dumas ...,La tulipe noire,1830,Décadence,Alexandre Dumas
...,...,...,...,...,...
109,Émile Zola ...,Le rêve,1885,"Décadence, Société et politique",Émile Zola
110,Émile Zola,Au bonheur des dames,1885,"Décadence, Société et politique",Émile Zola
111,Émile Zola,Nouveaux Contes à Ninon,1885,"Décadence, Société et politique",Émile Zola
112,Émile Zola ...,Contes à Ninon,1885,"Décadence, Société et politique",Émile Zola


In [271]:
df_books = df_books[df_books['Title'] != 'Nouveaux Contes à Ninon']
df_books.reset_index(drop=True, inplace=True)


In [277]:
df_books

,Author,Title,Year,Themes,Author_Cleaned
0,Alexandre Dumas ...,Pauline,1830,Décadence,Alexandre Dumas
1,Alexandre Dumas,Les trois mousquetaires,1830,Décadence,Alexandre Dumas
2,Alexandre Dumas ...,Gabriel Lambert,1830,Décadence,Alexandre Dumas
3,Alexandre Dumas ...,Le comte de Moret,1830,Décadence,Alexandre Dumas
4,Alexandre Dumas ...,La tulipe noire,1830,Décadence,Alexandre Dumas
...,...,...,...,...,...
91,Émile Zola ...,La curée,1885,"Décadence, Société et politique",Émile Zola
92,Émile Zola ...,Le rêve,1885,"Décadence, Société et politique",Émile Zola
93,Émile Zola,Au bonheur des dames,1885,"Décadence, Société et politique",Émile Zola
94,Émile Zola ...,Contes à Ninon,1885,"Décadence, Société et politique",Émile Zola


In [280]:
df_books.to_csv('final_list', index=False)

In [285]:
df_books = pd.read_csv('final_list')
df_books

,Author,Title,Year,Themes,Author_Cleaned
0,Alexandre Dumas ...,Pauline,1830,Décadence,Alexandre Dumas
1,Alexandre Dumas,Les trois mousquetaires,1830,Décadence,Alexandre Dumas
2,Alexandre Dumas ...,Gabriel Lambert,1830,Décadence,Alexandre Dumas
3,Alexandre Dumas ...,Le comte de Moret,1830,Décadence,Alexandre Dumas
4,Alexandre Dumas ...,La tulipe noire,1830,Décadence,Alexandre Dumas
...,...,...,...,...,...
91,Émile Zola ...,La curée,1885,"Décadence, Société et politique",Émile Zola
92,Émile Zola ...,Le rêve,1885,"Décadence, Société et politique",Émile Zola
93,Émile Zola,Au bonheur des dames,1885,"Décadence, Société et politique",Émile Zola
94,Émile Zola ...,Contes à Ninon,1885,"Décadence, Société et politique",Émile Zola


In [286]:
df_books = pd.read_csv('mapping1')
df_books

,Author,Title,Year,Themes
0,Honoré de Balzac,"La Comédie humaine, Études de mœurs",1800,Décadence
1,Honoré de Balzac,"La Comédie humaine, Études philosophiques et É...",1800,Décadence
2,Victor Hugo,Œuvres complètes de Victor Hugo,1800,"Décadence, Roman, Coutumes"
3,Honoré de Balzac,"La Comédie humaine, Études philosophiques,",1800,Décadence
4,Honoré de Balzac,La Comédie humaine,1800,Décadence
...,...,...,...,...
271,Stendhal ...,Les Cenci,1800,"Décadence, Critique, Roman, Coutumes"
272,Jules Verne,Le Tour du Monde en 80 Jours,1872,"Décadence, Fiction historique, Aventures, Tech..."
273,Jules Verne,De La Terre a La Lune,1872,"Décadence, Fiction historique, Aventures, Tech..."
274,Stendhal ...,Le Rouge et le Noir,1800,"Décadence, Critique, Roman, Coutumes"


In [287]:
df_books['Author_Cleaned'] = df_books['Author'].apply(lambda x: x.rstrip())
df_books

,Author,Title,Year,Themes,Author_Cleaned
0,Honoré de Balzac,"La Comédie humaine, Études de mœurs",1800,Décadence,Honoré de Balzac
1,Honoré de Balzac,"La Comédie humaine, Études philosophiques et É...",1800,Décadence,Honoré de Balzac
2,Victor Hugo,Œuvres complètes de Victor Hugo,1800,"Décadence, Roman, Coutumes",Victor Hugo
3,Honoré de Balzac,"La Comédie humaine, Études philosophiques,",1800,Décadence,Honoré de Balzac
4,Honoré de Balzac,La Comédie humaine,1800,Décadence,Honoré de Balzac
...,...,...,...,...,...
271,Stendhal ...,Les Cenci,1800,"Décadence, Critique, Roman, Coutumes",Stendhal
272,Jules Verne,Le Tour du Monde en 80 Jours,1872,"Décadence, Fiction historique, Aventures, Tech...",Jules Verne
273,Jules Verne,De La Terre a La Lune,1872,"Décadence, Fiction historique, Aventures, Tech...",Jules Verne
274,Stendhal ...,Le Rouge et le Noir,1800,"Décadence, Critique, Roman, Coutumes",Stendhal


In [288]:
# Comme La Comédie humaine est l'oeuvre de Balzac est non pas un livre, on le retire de la base de données
df_books = df_books[df_books['Author_Cleaned'] != "Honoré de Balzac"]
df_books['Author_Cleaned'] = df_books['Author_Cleaned'].apply(lambda x: x.rstrip())
# Supprimer les lignes où le titre commence par 'Œuvres complètes'
df_books = df_books[~df_books['Title'].str.startswith(('Œuvres complètes', 'Œuvres', 'Oeuvres', 'Poésies Complètes', 'Ouvres'))]
df_books = df_books[df_books['Year'] < 1900]
df_books = remove_similar_books_from_df(df_books)
df_books['Title'] = df_books['Title'].apply(lambda x: x.split('.')[0])
df_books = df_books.drop_duplicates(subset='Title')
df_books = df_books[df_books['Title'] != 'Le Rhin I']
df_books = df_books[~df_books['Title'].str.contains('\?', regex=True)]
df_books['Title'] = df_books['Title'].apply(lambda x: x.split(' of')[0])
df_books = df_books[df_books['Title'] != 'Nouveaux Contes à Ninon']


# Étape 1: Compter le nombre de livres par auteur
author_counts = df_books['Author_Cleaned'].value_counts()
print(author_counts) ####### TODO on remarque que Sand a 48 livres dans le DF soit biais 
# Étape 2: Filtrer les auteurs ayant plus de 2 livres
authors_with_more_than_two_books = author_counts[author_counts > 2].index

# Étape 3: Filtrer et garder les 2 premiers livres par auteur
df_books = (
    df_books[df_books['Author_Cleaned'].isin(authors_with_more_than_two_books)]
    .sort_values(['Author_Cleaned', 'Year'])  # Trier par auteur et par année ou autre critère
    .groupby('Author_Cleaned')
    .head(10)  # Garder les 10 premiers par groupe
)
df_books.reset_index(drop=True, inplace=True)

df_books

Author_Cleaned
George Sand                                        45
Jules Verne                                        33
Alexandre Dumas                                    25
Guy de Maupassant                                  15
Victor Hugo                                        10
Voltaire                                            9
Émile Zola                                          8
Alphonse Daudet                                     8
Théophile Gautier                                   7
Gustave Flaubert                                    6
Stendhal                                            5
Alexandre Dumas, Pere                               3
Edmond de Goncourt                                  3
Edmond de Goncourt, Jules de Goncourt               2
Paul Scarron                                        2
Eugène Sue                                          2
Octave Mirbeau                                      2
Jean-Jacques Rousseau                               1
Théophile Gau

<>:11: SyntaxWarning: invalid escape sequence '\?'
<>:11: SyntaxWarning: invalid escape sequence '\?'
/tmp/ipykernel_6929/199226954.py:11: SyntaxWarning: invalid escape sequence '\?'
  df_books = df_books[~df_books['Title'].str.contains('\?', regex=True)]
/tmp/ipykernel_6929/199226954.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_books['Author_Cleaned'] = df_books['Author_Cleaned'].apply(lambda x: x.rstrip())


,Author,Title,Year,Themes,Author_Cleaned
0,Alexandre Dumas ...,Pauline,1830,Décadence,Alexandre Dumas
1,Alexandre Dumas,Les trois mousquetaires,1830,Décadence,Alexandre Dumas
2,Alexandre Dumas ...,Gabriel Lambert,1830,Décadence,Alexandre Dumas
3,Alexandre Dumas ...,Le comte de Moret,1830,Décadence,Alexandre Dumas
4,Alexandre Dumas ...,La tulipe noire,1830,Décadence,Alexandre Dumas
...,...,...,...,...,...
94,Émile Zola ...,La curée,1885,"Décadence, Société et politique",Émile Zola
95,Émile Zola ...,Le rêve,1885,"Décadence, Société et politique",Émile Zola
96,Émile Zola,Au bonheur des dames,1885,"Décadence, Société et politique",Émile Zola
97,Émile Zola ...,Contes à Ninon,1885,"Décadence, Société et politique",Émile Zola


In [289]:
df_books.to_csv('liste_finale_cohérence_code', index=False)

In [290]:
df_books = pd.read_csv('liste_finale_cohérence_code')
df_books

,Author,Title,Year,Themes,Author_Cleaned
0,Alexandre Dumas ...,Pauline,1830,Décadence,Alexandre Dumas
1,Alexandre Dumas,Les trois mousquetaires,1830,Décadence,Alexandre Dumas
2,Alexandre Dumas ...,Gabriel Lambert,1830,Décadence,Alexandre Dumas
3,Alexandre Dumas ...,Le comte de Moret,1830,Décadence,Alexandre Dumas
4,Alexandre Dumas ...,La tulipe noire,1830,Décadence,Alexandre Dumas
...,...,...,...,...,...
94,Émile Zola ...,La curée,1885,"Décadence, Société et politique",Émile Zola
95,Émile Zola ...,Le rêve,1885,"Décadence, Société et politique",Émile Zola
96,Émile Zola,Au bonheur des dames,1885,"Décadence, Société et politique",Émile Zola
97,Émile Zola ...,Contes à Ninon,1885,"Décadence, Société et politique",Émile Zola
